In [102]:
import pandas as pd
import numpy as np
import json
import ijson
import requests
from bs4 import BeautifulSoup
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [91]:
#podatki iz jsona

file_path = "./News_Category_Dataset_IS_course.json"
data = [json.loads(line, object_hook=lambda o: str(o) if isinstance(o, (str, None))else o) for line in open(file_path, 'r')]

data

In [92]:
#naredi dataframe
df = pd.DataFrame(data)

categories = df['category'].value_counts()

num_categories = len(categories)
print(categories)

category
POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
Name: count, dtype: int64


In [98]:
# adds the whole story to the dataframe

def add_story(df):
    text_column = []

    for i in range(15360, len(df)):
        # print(f"index: {i}")
        short_description = df['short_description'].iloc[i]

        if (type(short_description) != str):
            # print(short_description)
            print(f"index {i}")
            link = df['link'].iloc[i]
            try:
                response = requests.get(link)
                
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')

                    # section = soup.find('section')
                    cur_text_arr = []
                    all_data_article = soup.find_all('div', class_='primary-cli') #all the text data in article that could be useful (the last few aren't)
                    for i in range(len(all_data_article) - 2):
                        k = all_data_article[i]
                        cur_text_arr.append(k.text)
                    current_string = " ".join(cur_text_arr)

                    df.at[i, 'short_description'] = current_string
            
            except:
                continue
                # text_column.append(current_string)

    

    # df['story'] = text_column
    return df




# links

stories = add_story(df)

stories


index 15366
index 15368
index 15487
index 15553
index 15607
index 15702
index 15877
index 16092
index 16265
index 16504
index 16529
index 16557
index 16693
index 17139
index 17211
index 17553
index 17575
index 18575
index 18766
index 18781
index 19105
index 19190
index 19383
index 19717
index 19785
index 19848
index 20138
index 20139
index 20142
index 20673
index 20682
index 20963
index 21551
index 21620
index 21675
index 21710
index 21868
index 21911
index 21958
index 22198
index 22248
index 23273
index 23286
index 23439
index 23516
index 23916
index 24189
index 24263
index 24312
index 24397
index 24458
index 24735
index 24872
index 25331
index 25614
index 25652
index 25710
index 25846
index 26076
index 26525
index 26817
index 27414
index 27459
index 27468
index 27642
index 27872
index 28346
index 28534
index 28622
index 28801
index 28810
index 29172
index 29188
index 29197
index 29212
index 29251
index 29352
index 29394
index 30028
index 30121
index 30248
index 30304
index 30478
inde

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,1663891200000
1,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,“Late Night” host Seth Meyers thinks he knows ...,Caroline Bologna,1663891200000
2,https://www.huffpost.com/entry/dodgers-basebal...,"Maury Wills, Base-Stealing Shortstop For Dodge...",SPORTS,WASHINGTON (Reuters) - The U.S. Supreme Court ...,"Beth Harris, AP",1663632000000
3,https://www.huffpost.com/entry/golden-globes-r...,Golden Globes Returning To NBC In January Afte...,ENTERTAINMENT,As we were stocking up on woven shoes and flat...,,1663632000000
4,https://www.huffpost.com/entry/biden-us-forces...,Biden Says U.S. Forces Would Defend Taiwan If ...,POLITICS,Michael B. Jordan has been knocking on doors i...,,1663545600000
...,...,...,...,...,...,...
148117,https://www.huffingtonpost.com/entry/girl-with...,'Girl With the Dragon Tattoo' India Release Ca...,ENTERTAINMENT,"""Sony Pictures will not be releasing The Girl ...",,1327708800000
148118,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,1327708800000
148119,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,1327708800000
148120,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,1327708800000


In [99]:
stories.to_csv("fixed_data.csv", index=False)

In [108]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()



# Function to preprocess text
def preprocess_text(text):
    # Tokenize the text into words
    text = str(text)
    words = word_tokenize(text.lower())  # Convert text to lowercase

    # Remove punctuation
    table = str.maketrans('', '', string.punctuation)
    words = [word.translate(table) for word in words if word.isalpha()]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Stemming (uncomment if you want to use stemming)
    stemmed_words = [stemmer.stem(word) for word in words]

    # Join the words back into a string
    preprocessed_text = ' '.join(lemmatized_words)
    return preprocessed_text

df = pd.read_csv('./fixed_data.csv', sep=',')
# df
df['cleaned_text'] = df['short_description'].apply(preprocess_text)

In [110]:
df.to_csv("fixed_data.csv", index=False)

In [ ]:
#### vectorization